In [ ]:
import onnx
from onnx_tf.backend import prepare

# ONNXモデルを読み込み
onnx_model = onnx.load('model_EnD_best.onnx')

# TensorFlowモデルに変換
tf_rep = prepare(onnx_model)

# TensorFlowモデルを保存
tf_rep.export_graph('tf_model_EnD')


INFO:absl:Function `__call__` contains input name(s) x with unsupported characters which will be renamed to transpose_14_x in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/usr24/e_haruki/e_haruki/sound-surveillance/convert_model/model/best_20241126/tf_model_EnD/assets


INFO:tensorflow:Assets written to: /home/usr24/e_haruki/e_haruki/sound-surveillance/convert_model/model/best_20241126/tf_model_EnD/assets
INFO:absl:Writing fingerprint to /home/usr24/e_haruki/e_haruki/sound-surveillance/convert_model/model/best_20241126/tf_model_EnD/fingerprint.pb


In [ ]:
import tensorflow as tf
# TensorFlow SavedModelディレクトリを指定してTFLiteConverterを作成
converter = tf.lite.TFLiteConverter.from_saved_model('tf_model_EnD')

# TFLiteモデルに変換
tflite_model = converter.convert()

# TFLiteモデルをファイルに保存
with open('tflite_model_EnD', 'wb') as f:
    f.write(tflite_model)


2024-11-26 11:41:41.979159: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-11-26 11:41:41.979184: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-11-26 11:41:41.981740: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/usr24/e_haruki/e_haruki/sound-surveillance/convert_model/model/best_20241126/tf_model_EnD
2024-11-26 11:41:41.982312: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-11-26 11:41:41.982322: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/usr24/e_haruki/e_haruki/sound-surveillance/convert_model/model/best_20241126/tf_model_EnD
2024-11-26 11:41:41.983185: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-11-26 11:41:41.989461: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path:

In [ ]:
# 動的量子化を有効にする
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# TFLiteモデルに変換
tflite_quant_model = converter.convert()

# 量子化されたTFLiteモデルを保存
with open("model_quant_dynamic_predecessor.tflite", "wb") as f:
    f.write(tflite_quant_model)

2024-11-26 11:44:31.151891: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-11-26 11:44:31.151913: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-11-26 11:44:31.152106: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /home/usr24/e_haruki/e_haruki/sound-surveillance/convert_model/model/baseline/tf_model_predecessor
2024-11-26 11:44:31.152434: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-11-26 11:44:31.152443: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /home/usr24/e_haruki/e_haruki/sound-surveillance/convert_model/model/baseline/tf_model_predecessor
2024-11-26 11:44:31.153046: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-11-26 11:44:31.158106: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at

In [ ]:
import os

# Path to the quantized model
quant_model_path = "model_quant_dynamic_EnD.tflite"

# Get the size of the model in bytes
model_size_bytes = os.path.getsize(quant_model_path)

# Convert to kilobytes (KB)
model_size_kb = model_size_bytes / 1024

# Convert to megabytes (MB)
model_size_mb = model_size_kb / 1024

print(f"Model size: {model_size_bytes} bytes")
print(f"Model size: {model_size_kb:.2f} KB")
print(f"Model size: {model_size_mb:.2f} MB")


Model size: 138488 bytes
Model size: 135.24 KB
Model size: 0.13 MB


In [ ]:
import tensorflow as tf
import numpy as np
import time

quant_model_path = "model_quant_dynamic_predecessor.tflite"

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=quant_model_path)

# Allocate tensors
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Create dummy input data
input_data = np.random.random(size=input_details[0]['shape']).astype(input_details[0]['dtype'])
interpreter.set_tensor(input_details[0]['index'], input_data)

# Measure inference time for each operation
print("Running inference with manual timing...")
start_time = time.time()
interpreter.invoke()
end_time = time.time()

print(f"\nTotal Inference Time: {(end_time - start_time) * 1000:.2f} ms")

# Get tensor details to understand intermediate tensors
print("\n=== Tensor Details ===")
for tensor in interpreter.get_tensor_details():
    print(f"Tensor name: {tensor['name']}")
    print(f"Shape: {tensor['shape']}")
    print(f"Data type: {tensor['dtype']}")
    print("-" * 50)


Running inference with manual timing...

Total Inference Time: 0.50 ms

=== Tensor Details ===
Tensor name: serving_default_input:0
Shape: [ 1  1 40 59]
Data type: <class 'numpy.float32'>
--------------------------------------------------
Tensor name: Const_4
Shape: [8]
Data type: <class 'numpy.float32'>
--------------------------------------------------
Tensor name: Const_2
Shape: [8]
Data type: <class 'numpy.float32'>
--------------------------------------------------
Tensor name: convolution
Shape: [ 8  5 11  1]
Data type: <class 'numpy.float32'>
--------------------------------------------------
Tensor name: depthwise
Shape: [1 3 3 8]
Data type: <class 'numpy.float32'>
--------------------------------------------------
Tensor name: transpose_2/perm
Shape: [4]
Data type: <class 'numpy.int32'>
--------------------------------------------------
Tensor name: Const
Shape: [3]
Data type: <class 'numpy.float32'>
--------------------------------------------------
Tensor name: Prod_1
Shape:

In [ ]:
quant_model_path = "model_quant_dynamic_EnD.tflite"

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path=quant_model_path)

# Allocate tensors
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Create dummy input data
input_data = np.random.random(size=input_details[0]['shape']).astype(input_details[0]['dtype'])
interpreter.set_tensor(input_details[0]['index'], input_data)

# Measure inference time for each operation
print("Running inference with manual timing...")
start_time = time.time()
interpreter.invoke()
end_time = time.time()

print(f"\nTotal Inference Time: {(end_time - start_time) * 1000:.2f} ms")

# Get tensor details to understand intermediate tensors
print("\n=== Tensor Details ===")
for tensor in interpreter.get_tensor_details():
    print(f"Tensor name: {tensor['name']}")
    print(f"Shape: {tensor['shape']}")
    print(f"Data type: {tensor['dtype']}")
    print("-" * 50)


Running inference with manual timing...

Total Inference Time: 2.47 ms

=== Tensor Details ===
Tensor name: serving_default_input:0
Shape: [  1   1  30 101]
Data type: <class 'numpy.float32'>
--------------------------------------------------
Tensor name: Const_4
Shape: [1]
Data type: <class 'numpy.int32'>
--------------------------------------------------
Tensor name: Const_2
Shape: [4 2]
Data type: <class 'numpy.int32'>
--------------------------------------------------
Tensor name: Const_1
Shape: [4 2]
Data type: <class 'numpy.int32'>
--------------------------------------------------
Tensor name: Const
Shape: [4 2]
Data type: <class 'numpy.int32'>
--------------------------------------------------
Tensor name: flatten/Reshape/shape/1
Shape: []
Data type: <class 'numpy.int32'>
--------------------------------------------------
Tensor name: strided_slice_1/stack_2
Shape: [1]
Data type: <class 'numpy.int32'>
--------------------------------------------------
Tensor name: Prod_1
Shape:

In [ ]:
import os

# Path to the quantized model
quant_model_path = "model_quant_dynamic_1026.tflite"

# Get the size of the model in bytes
model_size_bytes = os.path.getsize(quant_model_path)

# Convert to kilobytes (KB)
model_size_kb = model_size_bytes / 1024

# Convert to megabytes (MB)
model_size_mb = model_size_kb / 1024

print(f"Model size: {model_size_bytes} bytes")
print(f"Model size: {model_size_kb:.2f} KB")
print(f"Model size: {model_size_mb:.2f} MB")


Model size: 138488 bytes
Model size: 135.24 KB
Model size: 0.13 MB
